<a href="https://colab.research.google.com/github/shubhamksingh1/WordTagging/blob/main/SimilarityMatrix/Similarity_DEMO_Word_Tagging_Webex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library Imports

In [ ]:
import pandas as pd
import numpy as np

### Dataset Exploration

In [ ]:
df = pd.read_csv("/content/drive/My Drive/updated_dataset_ner.csv")

In [ ]:
df.shape

(499, 3)

In [ ]:
df.head()

,SentenceID,Words,Tags
0,1,Pankaj Seth,person
1,1,MS Excel,tool_skills
2,1,Auto ARIMA Mode,tech_skills
3,1,logistic Regression,tech_skills
4,1,Sentiment Analysis,tech_skills


In [ ]:
df.tail()

,SentenceID,Words,Tags
494,36,JAVA,tech_skills
495,36,TCS,organization
496,36,2,exp_in_months
497,36,navneet.kaur7@tcs.com,contact_email
498,36,8054090200,contact_no


In [ ]:
df.describe(include='all')

,SentenceID,Words,Tags
count,499.000000,499,499
unique,NaN,193,8
top,NaN,TCS,tech_skills
freq,NaN,35,226
mean,18.358717,NaN,NaN
std,10.892021,NaN,NaN
min,1.000000,NaN,NaN
25%,8.000000,NaN,NaN
50%,19.000000,NaN,NaN
75%,28.000000,NaN,NaN


In [ ]:
df.drop(['SentenceID'],inplace=True,axis=1)

In [ ]:
df.head()

,Words,Tags
0,Pankaj Seth,person
1,MS Excel,tool_skills
2,Auto ARIMA Mode,tech_skills
3,logistic Regression,tech_skills
4,Sentiment Analysis,tech_skills


In [ ]:
d1 = df.groupby(df['Tags'])['Words'].apply(','.join).reset_index()

In [ ]:
d1

,Tags,Words
0,contact_email,"pankaj.seth@tcs.com,deepna.cp@tcs.com,debankur..."
1,contact_no,"9173685598,9400521890,8981532992,9887770063,78..."
2,exp_in_months,"28,2,6,4,5,2,3,5,2,2,4,2,4,4,5,5,5,5,4,5,5,5,5..."
3,exp_in_yrs,"8,4,3,3,8,1,1,1.5,2,1.5,1"
4,organization,"TCS,TCS,TCS,TCS,TCS,TCS,TCS,TCS,TCS,TCS,TATA C..."
5,person,"Pankaj Seth,DEEPNA CP,DEBANKUR SAHA,Shruti Mah..."
6,tech_skills,"Auto ARIMA Mode,logistic Regression,Sentiment ..."
7,tool_skills,"MS Excel,Power BI,Qlik Sense,SAS,Eclipse,Git,V..."


In [ ]:
def cleaning_text(inp):
    corpus = []
    reply = str(inp).strip()
    reply = reply.lower()
    reply = reply.split(",")
    reply = list(set(reply))
    return reply

In [ ]:
d1['Words'] = d1['Words'].apply(lambda x: cleaning_text(x))

In [ ]:
d1

,Tags,Words
0,contact_email,"[ishank.goel@tcs.com, niki.pat@tcs.com, mahesh..."
1,contact_no,"[9400521890, 8148223880, 8445340053, 805409020..."
2,exp_in_months,"[5, 3, 4, 28, 2, 6]"
3,exp_in_yrs,"[1.5, 8, 3, 4, 1, 2]"
4,organization,"[tata consultancy services ltd, tcs]"
5,person,"[pankaj seth, akanksha deshmukh, kuppam ezra n..."
6,tech_skills,"[hadoop, , angular2, auto arima mode, python3,..."
7,tool_skills,"[cognos bi, visual studio, octave, power bi, n..."


### Exact Match Word Tagging If the word is in corpus

In [ ]:
inp = input("Enter the value : ")

Enter the value : Google Cloud Platform


In [ ]:
val_to_search = inp.lower().strip()

In [ ]:
found = 0

In [ ]:
for i in range(d1.shape[0]):
    if val_to_search in d1['Words'][i]:
        found=1
        print(d1['Tags'][i])
        break;

tool_skills


In [ ]:
found

1

### If the input is not exactly same as in corpus

### Importing Libraries

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def compute_similarity(str1,str2):
    train = [str1,str2]
    #Define a Count Vectorizer Object
    countvector = CountVectorizer()

    #Construct the required TF matrix by fitting and transforming the data
    count_matrix = countvector.fit_transform(train)

    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    return cosine_sim

In [ ]:
arrlist = [0 for i in range(d1.shape[0])]

In [ ]:
df_cos_sim = pd.DataFrame(np.array([arrlist]),columns=d1['Tags'].values)
df_cos_sim = df_cos_sim.fillna(0)

In [ ]:
df_cos_sim

,contact_email,contact_no,exp_in_months,exp_in_yrs,organization,person,tech_skills,tool_skills
0,0,0,0,0,0,0,0,0


In [ ]:
inp = input("Enter the value : ")

Enter the value : Google Cloud


In [ ]:
val_to_search = inp.lower().strip()

In [ ]:
for i in range(len(d1['Tags'])):
    set_A = d1['Words'][i]
    catA_string=' '.join(set_A)
    catInp = val_to_search
    similarity = compute_similarity(catA_string,catInp)[0][1]
    similarity = similarity * 100
    if df_cos_sim[0:1][d1['Tags'][i]][0]== 0:
            df_cos_sim[0:1][d1['Tags'][i]][0]= similarity


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_cos_sim

,contact_email,contact_no,exp_in_months,exp_in_yrs,organization,person,tech_skills,tool_skills
0,0,0,0,0,0,0,0,19


In [ ]:
result = df_cos_sim.idxmax(axis=1)[0]

In [ ]:
result

'tool_skills'

### Now we will make all these as modular and if the exact word is not in corpus then only we will apply the similarity algorithm

In [ ]:
def find_tag(inp,d1=d1):
    val_to_search = inp.lower().strip()
    found = 0
    for i in range(d1.shape[0]):
        if val_to_search in d1['Words'][i]:
            found=1
            result = d1['Tags'][i]
            break;
    if found == 1:
        return result
    arrlist = [0 for i in range(d1.shape[0])]
    df_cos_sim = pd.DataFrame(np.array([arrlist]),columns=d1['Tags'].values)
    df_cos_sim = df_cos_sim.fillna(0)
    for i in range(len(d1['Tags'])):
        set_A = d1['Words'][i]
        catA_string=' '.join(set_A)
        catInp = val_to_search
        similarity = compute_similarity(catA_string,catInp)[0][1]
        similarity = similarity * 100
        if df_cos_sim[0:1][d1['Tags'][i]][0]== 0:
                df_cos_sim[0:1][d1['Tags'][i]][0]= similarity
    result = df_cos_sim.idxmax(axis=1)[0]
    return result

In [ ]:
inp = input("Enter the string to find tag : \n")

Enter the string to find tag : 
pyon


In [ ]:
result = find_tag(inp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
result

'contact_email'